# 1. RAFT training data generation using GPT-4o

In this notebook, we'll synthesize some training data that will eventually be used to fine-tune a GPT-4o mini model in order to adapt it to a set of document(s) and specific domain. **This step is critical as the quality of your training data will greatly influence the performance of your fine-tuned model.**

For RAFT, these are the different steps to prepare the training dataset:
- Collect Domain-Specific Documents: Gather documents relevant to the domain you want to specialize the LLM in (e.g., medical documents for PubMed, legal documents, API documentation for software).
- Chunk the file into Documents
- For each Document chunk, generate a set of Questions that can be answered from the Document
- For each Document-Question pair, create a list of documents using:
    - **Golden Document (D*)**: Document that contains the answer to the question.
    - **Distractor Documents (Dk)**: Documents that do not contain relevant information.
- Question-Answer-Document Triplets: From each **Document-Question** pair, generate a factual **Answer** based on the Golden Document.

Curating a good training dataset often involves manual work and review by SMEs. That said, we can use an LLM to help us generate an initial set of training examples that can be vetted and further refined by SMEs.


### 0. Pre-requisites

1. Create a code environment and install the necessary packages

```shell
conda env create -n raft python=3.10

conda activate raft

pip install -r requirements.txt
```

2. Create a .env file based on the sample.env file in this repository. Note: You'll need a GPT-4o and a GPT-4o mini deployment

In [66]:
from openai import AzureOpenAI
from langchain_experimental.text_splitter import SemanticChunker
from langchain_openai.embeddings import AzureOpenAIEmbeddings
from dotenv import load_dotenv
from io import BytesIO
import base64
from typing import Literal, Any
import os
from math import ceil
import random
from tqdm import tqdm
from pdf2image import convert_from_path


load_dotenv()


### 1. Loading and chunking domain-specific documents

For Retrieval Augmented Fine Tuning, we need to generate Question-Documents-Answer triplets. The first step is to create document chunks based on our domain-specific documents we want to specialize our model on.

For this workshop, we will use the publicly available [BMO Better Banking Guide](./data/better_banking_guide_en.pdf)

The guide contains information about various banking accounts offered by BMO as well as how-to guides on e.g how to access Account statements etc.

The document is in PDF format and contains a number of tables and charts, we will use GPT-4o to convert the pages content to markdown.

**a. First we'll need to convert the document pages to images encoded in base64**

In [139]:
def pdf_to_base64_urls(pdf_path):
  """Converts each page of a PDF to a base64 encoded URL starting with 'data:image/jpeg'.

  Args:
    pdf_path: Path to the PDF file.

  Returns:
    A list of base64 encoded image URLs, one for each page.
  """

  images = convert_from_path(pdf_path)
  base64_urls = []

  for image in images:
    img_byte_arr = BytesIO()
    image.save(img_byte_arr, format="JPEG")
    img_byte_arr.seek(0)
    base64_encoded = base64.b64encode(img_byte_arr.read()).decode('utf-8')
    base64_url = f"data:image/jpeg;base64,{base64_encoded}"
    base64_urls.append(base64_url)

  return base64_urls

pdf_path = 'data/better_banking_guide_en.pdf'
image_data = pdf_to_base64_urls(pdf_path)

**b. Now we can call GPT-4o to convert our images to markdown**

In [145]:
def gpt_image_to_markdown(image_data, client):
    """
    Converts an image to markdown using GPT-4o.
    
    Args:
    image_data: A base64 encoded image.
    client: An AzureOpenAI client

    Returns:
    A list of base64 encoded image URLs, one for each page.
    """
    messages = [
        {"role":"system", "content":"""You are an AI image assistant capable of extracting text from images
         Given an image, you must extract the any visible text on the image and return it in Markdown.
         You must keep the original layout and formatting of the text as much as possible in Markdown format.
         Pay attention to the text size and use headers, subheaders, bold, italic, tables etc where necessary."""},
         {"role":"user", "content":[{
                "type":"image_url",
                "image_url":{
                    "url":image_data
                    }
         }]}
    ]
    try:
        response = client.chat.completions.create(
            model="gpt-4o-global",
            messages=messages
        )
        
        return response.choices[0].message.content
    
    except Exception as e:
        print(f"Error: {e}")
        return None

client = AzureOpenAI(
    azure_endpoint=os.getenv("AZURE_OPENAI_API_ENDPOINT"),
    api_version="2024-02-01",
    api_key=os.getenv("AZURE_OPENAI_API_KEY")
)

markdown_doc = ""

with tqdm(total=len(image_data), desc="Converting images to Markdown") as pbar:
    for img_data in image_data:
        result = gpt_image_to_markdown(img_data, client)
        markdown_doc += "\n" + result
        pbar.update(1)


Converting images to Markdown: 100%|██████████| 26/26 [04:34<00:00, 10.54s/it]


**c. Let's chunk the Markdown using Langchain**

We first use Langchain's `MarkdownHeaderTextSplitter` to split the document based on headers and then further split the chunks using `RecursiveCharacterTextSplitter` with a chunk size of 1024. Finally, we filter out any chunks that are too short to contain any valuable information.

In [219]:
from langchain_text_splitters import MarkdownHeaderTextSplitter, RecursiveCharacterTextSplitter
import re

def remove_special_characters(string):
    """
    Remove special characters from a string.
    
    Parameters:  
    string (str): The input string from which special characters need to be removed.  
  
    Returns:  
    str: A new string with special characters removed.
    """
    return re.sub(r'[^a-zA-Z0-9\s]', '', string)

headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2")    
]

markdown_splitter = MarkdownHeaderTextSplitter(
    headers_to_split_on=headers_to_split_on,
    strip_headers=False
    )

markdown_doc_splits = markdown_splitter.split_text(markdown_doc)
print(f"Number of chunks after markdown split: {len(markdown_doc_splits)}")

chunk_size = 1024
chunk_overlap = 50
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size, 
    chunk_overlap=chunk_overlap,
)


chunked_document = text_splitter.split_documents(markdown_doc_splits)

print(f"Number of chunks after markdown + recursive split: {len(chunked_document)}")

chunks = [chunk.page_content for chunk in chunked_document if len(remove_special_characters(chunk.page_content))>100]

print(f"Number of chunks after filtering out empty: {len(chunks)}")


Number of chunks after markdown split: 14
Number of chunks after markdown + recursive split: 107
Number of chunks after filtering out empty: 100


### 2. Generate training data from the chunked documents

We define 2 main functions to generate our Question-Document-Answer triplets from our chunked document

1. `generate_instructions_gen()`: This function generates a list of questions based on an input document chunk
2. `generate_label()`: This function generates an Answer based on a Question-Document chunk pair

**a. First, lets look at the `generate_instructions_gen()` function on a sample**

In [65]:
def strip_str(s: str) -> str:
    """
    Helper function for helping format strings returned by GPT-4o.
    
    Parameters:  
    s (str): The input string to be formatted.  
  
    Returns:  
    str: A formatted string 
    """
    l, r = 0, len(s)-1
    beg_found = False
    for i in range(len(s)):
        if s[i].isalpha():
            if not beg_found:
                l = i
                beg_found = True
            else:
                r = i 
    r += 2
    return s[l:min(r, len(s))]

def generate_instructions_gen(client: AzureOpenAI, chunk: Any, x: int = 5, model: str = None) -> list[str]:
    """
    Generates a list of questions or use cases based on a provided chunk of context using an Azure OpenAI model.  

    Parameters:  
    client (AzureOpenAI): An instance of the Azure OpenAI client used to communicate with the OpenAI API.  
    chunk (Any): The context or chunk of text based on which the questions are to be generated.  
    x (int, optional): The number of questions to generate. Default is 5.  
    model (str, optional): The specific model to use for generating the questions. Default is None, which uses the default model configured in the client.  
  
    Returns:  
    list[str]: A list of generated questions.
    """
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": "You are a synthetic question-answer pair generator. Given a chunk of context about some topic(s), generate exactly %s example questions a user could ask and would be answered using information from the chunk. For example, if the given context was a Wikipedia paragraph about the United States, an example question could be 'How many states are in the United States?'" % (x)},
            {"role": "system", "content": "The questions should be able to be answered in a few words or less. Include only the questions in your response."},
            {"role": "user", "content": str(chunk)}
        ]
    )

    queries = response.choices[0].message.content.split('\n')
    queries = [strip_str(q) for q in queries]
    queries = [q for q in queries if any(c.isalpha() for c in q)]
    return queries[:int(x)]

Let's visualize an example picked randomly from our Document chunks

In [ ]:
synthesis_model = os.getenv("AZURE_OPENAI_4o_DEPLOYMENT")

In [191]:
sample_index = random.randint(0, len(chunks)-1)
chunk = chunks[sample_index]

queries = generate_instructions_gen(client, chunk, x=5, model="gpt-4o-global")

In [192]:
print(chunk)

| Services                                    | Primary Chequing | Interest Chequing⁴ | Premium Rate Savings | Smart Saver⁴ |
|---------------------------------------------|------------------|--------------------|----------------------|--------------|
| **Self-serve**                              |                  |                    |                      |              |
| **Electronic (per item)**                   |                  |                    |                      |              |
| Pre-authorized bill payment/debit           | $1.00            | $1.00              | $1.00                | $5.00⁵       |
| Debit card purchase¹                        | $1.00            | $1.00              | Not available        | Not available|
| Interac Online debit                        | $1.00            | $1.00              | $1.00                | Not available|
| **ABM (per item)**                          |                  |                    |                      |        

In [232]:
queries


['How much does a pre-authorized bill payment/debit cost with a Primary Chequing account?',
 'What is the cost of a debit card purchase with an Interest Chequing account?',
 'Are debit card purchases available with a Premium Rate Savings account?',
 'How much does an Interac Online debit cost with a Smart Saver account?',
 'What is the fee for a pre-authorized bill payment/debit with a Smart Saver account?']

**b. Generating questions, answers and adding distractor documents** 

In [195]:
import datasets
from datasets import Dataset, load_dataset
import random
from typing import Any

def encode_question_gen(question: str, chunk: Any) -> list[str]:
    """
    Encode multiple prompt instructions into a single string for the general case (`pdf`, `json`, or `txt`).

    Parameters:  
    question (str): The question to be answered.  
    chunk (Any): The context or chunk of text that provides the information needed to answer the question.  
  
    Returns:  
    list[str]: A list of messages formatted for the language model API, including system and user roles.  
    """
    
    prompts = []
        
    prompt = """
        Question: {question}\n Context: {context}\n
        Answer this question using the information given in the context above and no prior knowledge. Here is things to pay attention to: 
        - First provide step-by-step reasoning on how to answer the question. 
        - In the reasoning, if you need to copy paste some sentences from the context, include them in ##begin_quote## and ##end_quote##. This would mean that things outside of ##begin_quote## and ##end_quote## are not directly copy paste from the context. 
        - End your response with final answer in the form <ANSWER>: $answer, the answer should be given in a joyful and friendly tone.
        - If the answer cannot be found in the context, say "I'm sorry, I cannot answer this question as I'm missing the required information"
        You MUST begin your final answer with the tag "<ANSWER>:".
    """.format(question=question, context=str(chunk))
    prompts.append({"role": "system", "content": "You are a helpful question answerer who can provide an answer given a question and relevant context."})
    prompts.append({"role": "user", "content": prompt})
    return prompts

def generate_label(client: AzureOpenAI, question: str, context: Any, model: str = None) -> str | None:
    """
    Generates the label / answer to `question` using `context` and GPT-4o.

    Parameters:  
    client (AzureOpenAI): An instance of the Azure OpenAI client used to communicate with the OpenAI API.  
    question (str): The question to be answered.  
    context (Any): The context or chunk of text that provides the information needed to answer the question.  
    model (str, optional): The specific model to use for generating the answer. Default is None, which uses the default model configured in the client.  
  
    Returns:  
    str | None: The generated answer from the language model, or None if no answer was generated.
    """
    question = encode_question_gen(question, context)
    response = client.chat.completions.create(
        model=model,
        messages=question,
        n=1,
        temperature=0
    )
    response = response.choices[0].message.content
    return response

def add_chunk_to_dataset(
    client: AzureOpenAI,
    chunks: list[str], 
    chunk: str, 
    x: int = 5, 
    num_distract: int = 3, 
    p: float = 0.8,
    model: str = None
) -> None:
    """
    Given a chunk, create {Q, A, D} triplets and add them to the dataset.

     Parameters:  
    client (AzureOpenAI): An instance of the Azure OpenAI client used to communicate with the OpenAI API.  
    chunks (list[str]): A list of chunks of text from which distractor documents can be sampled.  
    chunk (str): The chunk of text to use as the primary context for generating questions and answers.  
    x (int, optional): The number of questions to generate for the given chunk. Default is 5.  
    num_distract (int, optional): The number of distractor documents to include with each question. Default is 3.  
    p (float, optional): The probability of including the oracle (original) document as part of the context. Default is 0.8.  
    model (str, optional): The specific model to use for generating questions and answers. Default is None, which uses the default model configured in the client. 
    """
    global ds
    global errors
    i = chunks.index(chunk)
    try:
        qs = generate_instructions_gen(client, chunk, x, model)
    except Exception as e:
        errors.append(e)
        return None
    for q in qs:
        datapt = {
            "id": None,
            "type": None,
            "question": None,
            "context": None,
            "oracle_context": None,
            "cot_answer": None
        }

        datapt["id"] = f"seed_task_{i}"
        datapt["type"] = "general"
        datapt["question"] = q

        # add num_distract distractor docs
        docs = [chunk]
        indices = list(range(0, len(chunks)))
        indices.remove(i)
        for j in random.sample(indices, num_distract):
            docs.append(chunks[j])
        
        # decides whether to add oracle document
        oracle = random.uniform(0, 1) < p
        if not oracle:
            docs[0] = chunks[random.sample(indices, 1)[0]]
        random.shuffle(docs)

        d = {
            "title": [],
            "sentences": []
        }

        d["title"].append(["placeholder_title"]*(num_distract+1))
        d["sentences"].append(docs)
        datapt["context"] = d
        datapt["oracle_context"] = chunk

        # add answer to q
        try:
            datapt["cot_answer"] = generate_label(client, q, chunk, model=model)
        except Exception as e:
            errors.append(e)
            continue

        # construct model instruction 
        context = ""
        for doc in docs:
            context += "<DOCUMENT>" + str(doc) + "</DOCUMENT>\n"
        context += q
        datapt["instruction"] = context

        # add to dataset
        if not ds:
            # init ds
            datapt["id"] = [datapt["id"]]
            datapt["type"] = [datapt["type"]]
            datapt["question"] = [datapt["question"]]
            datapt["context"] = [datapt["context"]]
            datapt["oracle_context"] = [datapt["oracle_context"]]
            datapt["cot_answer"] = [datapt["cot_answer"]]
            datapt["instruction"] = [datapt["instruction"]]
            ds = Dataset.from_dict(datapt)
        else:
            ds = ds.add_item(datapt)

**Let's execute this function a in multi-threaded way to speed up the process**

In [220]:
from tqdm import tqdm

import concurrent.futures

errors = []
ds = Dataset.from_dict({})


def process_chunk(chunk):
    add_chunk_to_dataset(client, chunks, chunk, 5, 3, model="gpt-4o-global")

# Create a ThreadPoolExecutor with the desired number of workers
with concurrent.futures.ThreadPoolExecutor() as executor:
    # Submit the tasks to the executor and store the Future objects
    futures = [executor.submit(process_chunk, chunk) for chunk in chunks]

    # Use tqdm to create a progress bar
    with tqdm(total=len(chunks), desc="Processing chunks") as pbar:
        # Iterate over the completed futures as they become available
        for future in concurrent.futures.as_completed(futures):
            # Get the result of the completed future
            result = future.result()
            # Update the progress bar
            pbar.update(1)

# Print any errors that occurred during processing
print(f'Number of processing errors: {errors}/{len(chunks)}')

Processing chunks: 100%|██████████| 100/100 [02:20<00:00,  1.40s/it]

Number of processing errors: []/100


In [235]:
training_df = ds.to_pandas()

print(f'{training_df.shape[0]} rows and {training_df.shape[1]} columns in the training dataset')

477 rows and 7 columns in the training dataset


In [234]:
# Previewing the generated data

training_df.head(2)

,id,type,question,context,oracle_context,cot_answer,instruction,messages
0,seed_task_5,general,What is the minimum balance required for a Sma...,{'sentences': [['¹ Customer is responsible for...,| is available in U.S. dollars ...,To determine the minimum balance required for ...,<DOCUMENT>¹ Customer is responsible for all fe...,"[{'role': 'user', 'content': '<DOCUMENT>¹ Cust..."
1,seed_task_3,general,How many personal accounts does BMO Bank of Mo...,{'sentences': [['| Per Item | Per Item | | ---...,## Personal accounts \n### Step 1: Select you...,To determine how many personal accounts BMO Ba...,<DOCUMENT>| Per Item | Per Item |\n| --- | ---...,"[{'role': 'user', 'content': '<DOCUMENT>| Per ..."


**c. Formatting the data in chat format for fine tuning with Azure OpenAI**

The conversational chat format is required to fine-tune gpt-4o-mini

In [222]:
training_df["messages"] = training_df.apply(lambda x: [
                                                     {"role":"user", "content":x['instruction']},
                                                     {"role":"assistant", "content":x['cot_answer']}
                                                     ], axis=1)

In [225]:
training_df.messages.values[12]

[{'role': 'user',
  'content': "<DOCUMENT>You can reduce the impact to the environment!\n\nWe offer you the ability to stop receiving paper copies of your Everyday Banking statements for your Personal accounts2. Go to a branch to request the paperless option or, more conveniently, sign into Online Banking, select the Accounts and Plans tab followed by the Bank Statements tab and simply select the option to stop receiving paper copies of your Everyday Banking statements. If you are not already registered for Online Banking, be sure to register at bmo.com today.\n\nWhat’s more, you are now able to view your cheque images through Online Banking.\n\n---\n\n## Account Tracker\n\nBMO Account Tracker is a complementary money management service that provides you with a complete picture of your personal finances.\n\n---</DOCUMENT>\n<DOCUMENT>Of course, we're always here to help — you can contact us online, by phone or by visiting one of our branches.\n\n---\n\n¹ Some products and services are o

In [226]:
training_df.dropna(subset=['cot_answer'], inplace=True)

### 2. Spitting our data into training and test sets

Splitting your data into training and testing sets when fine-tuning a large language model (LLM) is crucial for ensuring the model's performance and generalization capabilities. The training set is used to teach the model, allowing it to learn patterns from the data. However, to objectively evaluate how well the model has learned and to ensure it can generalize to unseen data, a separate testing set is necessary. 

We will use this test set in order to measure the improvement of performance we get from using RAFT over RAG with gpt-4o-mini

In [228]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(training_df, test_size=0.2, random_state=42)

In [231]:

if not os.path.exists("data/training_data"):
    os.makedirs("data/training_data")

train_df.to_json("data/training_data/banking_train.jsonl", orient="records", lines=True)
test_df.to_json("data/training_data/banking_test.jsonl", orient="records", lines=True)

In [230]:
test_df.to_json("data/training_data/banking_test_with_metadata.json", orient="records", lines=True)